# 문제 6

[Kaggle 형] train_prob.csv로 failure 예측하는 모델을 만들고, 

test_prob.csv에 대한 failure가 1일 확률 예측하여 다음과 같은 형식의 answer6.csv를 만들어라. 

측정 지표는 AUC(area under of ROC curve)이다. id 는 테스트 케이스의 id 이고, failure에는 failure가 1이 될 확률이다.

id,failure

16115, 0.1

16116, 0.2


**강사: 멀티캠퍼스 강선구(sunku0316.kang@multicampus.com, sun9sun9@gmail.com)**

In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import sys
import xgboost as xgb

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels, xgb]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1
xgboost 0.80


In [2]:
# 시각화 모듈 설정
# 참고용 차트를 출력하기 위함

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib as mpl
import matplotlib.font_manager as fm
plt.rc('font', family='Malgun Gothic')
mpl.rcParams['axes.unicode_minus'] = False

In [3]:
df_train = pd.read_csv('train_prob.csv', index_col = ['id'])
df_test = pd.read_csv('test_prob.csv', index_col = ['id'])
df_ans = pd.read_csv('test_prob_ans.csv', index_col = ['id'])
s_ans = df_ans['failure']

In [4]:
df_train[['na_1', 'na_2']]  = df_train[['measurement_3', 'measurement_5']].isna()
df_test[['na_1', 'na_2']]  = df_test[['measurement_3', 'measurement_5']].isna()

In [5]:
# 방법 2 : train / test를 통합하여 처리해봅니다.

from sklearn.experimental import enable_iterative_imputer# 구문을 사용하여 실험 단계인 모듈을 활성화하고, 
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

X_imp = ['measurement_{}'.format(i) for i in range(3, 10)] + ['measurement_17']
# train에 등장하지 않은 수준이 있습니다, test를 포함하여 결측처리 모델을 만듭니다.
s_imp = pd.concat([
        df_train[X_imp + ['product_code']],
        df_test[X_imp + ['product_code']]
], axis=0).groupby('product_code')\
.apply(
    lambda x: IterativeImputer(estimator=LinearRegression(),random_state=123).fit(x[X_imp])
)
# train에 적용합니다.
df_train[X_imp] = df_train[X_imp + ['product_code']]\
            .groupby('product_code')\
            .apply(
                lambda x: pd.DataFrame(s_imp.loc[x.name].transform(x[X_imp]), index=x.index, columns=X_imp)
            )
# test에 적용합니다.
df_test[X_imp] = df_test[X_imp + ['product_code']]\
            .groupby('product_code')\
            .apply(
                lambda x: pd.DataFrame(s_imp.loc[x.name].transform(x[X_imp]), index=x.index, columns=X_imp)
            )
X_mean = ['measurement_{}'.format(i) for i in range(10, 17)]
# train에 등장하지 않은 범주를 처리하기 위해 합치니다.
df_mean = pd.concat([
            df_train[['product_code'] + X_mean],
            df_test[['product_code'] + X_mean]
        ]).groupby('product_code')[X_mean].agg('mean')

df_train[X_mean] = df_train.groupby('product_code')[X_mean]\
            .apply(lambda x: pd.DataFrame(x.fillna(df_mean.loc[x.name]), index=x.index, columns=x.columns))
df_test[X_mean] = df_test.groupby('product_code')[X_mean]\
            .apply(lambda x: pd.DataFrame(x.fillna(df_mean.loc[x.name]), index=x.index, columns=x.columns))

In [6]:
# test가 train에 등장하지 않은 product_code로 구성됨을 고려하면, 모델의 동작 환경은 
# 이미 테스트 케이스가 만들어진 상황에서 동작함을 생각해볼수가 있습니다.
# 이미 테스트 케이스가 있다는 상황을 가정하면 train / test에 있는 것까지 포함하여 처리과정을 구상할 수 있습니다.
m = pd.concat([
    df_train['loading'], df_test['loading']
], axis=0).mean()

df_train['loading'] = df_train['loading'].fillna(m)
df_test['loading'] = df_test['loading'].fillna(m)

df_train['loading_log'] = np.log(df_train['loading'])
df_test['loading_log'] = np.log(df_test['loading'])

In [7]:
X_all = df_test.columns.tolist()
np.array(X_all)
df_train.isna().sum().sum(), df_test.isna().sum().sum()

(0, 0)

- 문제 1: na_1, na_2 

- 문제 2: np.log('loading'), attribute_0, attribute_1: 도움이 안됩니다.

- 문제 3: LR + \['loading','measurement_1', 'measurement_4', 'measurement_14', 'measurement_17', 'na_1'\], (0.5838326230092876, 0.5936682060775388)

- 문제 4: LDA: 1. transform, 2. predict / LR + pca measurement_0~17, n_components = 7

- 문제 5: RF: ({'n_estimators': 15, 'max_depth': 7, 'min_samples_split': 512}, 0.5745226991354744) 

In [8]:
df_train['product_code'].value_counts()

C    5765
E    5343
B    5250
A    5100
Name: product_code, dtype: int64

In [9]:
df_test['product_code'].value_counts()

D    5112
Name: product_code, dtype: int64

In [10]:
X_all = df_test.columns.tolist()
np.array(X_all)

array(['product_code', 'loading', 'attribute_0', 'attribute_1',
       'attribute_2', 'attribute_3', 'measurement_0', 'measurement_1',
       'measurement_2', 'measurement_3', 'measurement_4', 'measurement_5',
       'measurement_6', 'measurement_7', 'measurement_8', 'measurement_9',
       'measurement_10', 'measurement_11', 'measurement_12',
       'measurement_13', 'measurement_14', 'measurement_15',
       'measurement_16', 'measurement_17', 'na_1', 'na_2', 'loading_log'],
      dtype='<U14')

In [11]:
from sklearn.model_selection import GroupKFold
gkf = GroupKFold(4)
for train_idx, valid_idx in gkf.split(df_train[X_all], df_train['failure'], groups=df_train['product_code']):
    print(
        df_train.iloc[train_idx]['product_code'].unique(), 
        df_train.iloc[valid_idx]['product_code'].unique()
    )

['A' 'B' 'E'] ['C']
['A' 'B' 'C'] ['E']
['A' 'C' 'E'] ['B']
['B' 'C' 'E'] ['A']


In [12]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# LR + ['loading','measurement_1', 'measurement_4', 'measurement_14', 'measurement_17', 'na_1']
gkf = GroupKFold(4)
ct = ColumnTransformer([
    ('std', StandardScaler(), ['loading','measurement_1', 'measurement_4', 'measurement_14', 'measurement_17']),
    ('pt', 'passthrough', ['na_1'])
])
clf_lr = make_pipeline(ct, LogisticRegression(solver = 'lbfgs'))
scores = cross_validate(
    clf_lr, df_train[X_all], df_train['failure'], groups = df_train['product_code'],
    cv = gkf, scoring='roc_auc', return_train_score=True
)
scores

{'fit_time': array([0.02490568, 0.01666737, 0.01682687, 0.02120805]),
 'score_time': array([0.00408864, 0.00457239, 0.00423026, 0.00372148]),
 'test_score': array([0.58821746, 0.58491734, 0.58894014, 0.59540058]),
 'train_score': array([0.59262252, 0.59350804, 0.59192438, 0.58956303])}

# Kaggle형 풀이 단계

Step 0: Kaggle용 데이터셋을 만든다.

Step 1: 검증 방법을 정하고, 검증 루틴을 만듭니다.

Step 2: Baseline 모델을 만듭니다

Step 3: 모델 선택 루틴을 만듭니다.

|id|failure|
|----|----|
|16115| 0.1|
|16116| 0.2|

....	

Step 4: 모델 개선 작업을 합니다.